<a href="https://colab.research.google.com/github/oo363636/project/blob/master/5008assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install dgl

In [ ]:
import numpy as np
import networkx as nx
import pickle as pkl
import dgl
from dgl import DGLGraph
from dgl.nn import GINConv
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn

In [ ]:
dataset = pkl.load(open('/content/drive/My Drive/Colab Notebooks/hw_dataset.pkl', 'rb'))
# np.set_printoptions(threshold=10)
print('labels:',dataset['labels'])        # length 3327
print('features:',dataset['features'])      # 3327 * 3703
print('train_mask:',(dataset['train_mask']))   # length 3327 
print('val_mask:',dataset['val_mask'])      # length 3327
print('source_nodes',dataset['source_nodes'])   # length 12431
print('target_nodes',dataset['target_nodes'])   # length 12431
print('nodes:',dataset['nodes'])         # length 3327
print('num_classes:',dataset['num_classes'])
# print(dataset)

labels: [4 1 5 ... 3 1 0]
features: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
train_mask: [False, False, False, False, True, False, False, True, False, True, False, True, False, False, False, True, True, False, True, False, False, True, False, False, False, False, False, False, False, True, False, False, True, False, True, False, False, True, False, True, False, False, False, False, False, True, False, True, False, True, True, False, False, False, False, True, True, False, True, False, True, False, True, False, False, False, False, False, False, True, True, True, True, False, False, False, False, True, False, True, True, True, True, True, False, False, True, False, False, False, False, False, True, False, True, False, False, True, True, False, False, False, True, False, False, False, False, False, True, True, True, True, False, True, False, False, False, False, False, False

In [ ]:
def build_graph():

  src = dataset['source_nodes']
  dst = dataset['target_nodes']
  u = np.concatenate([src, dst])
  v = np.concatenate([dst, src])
  
  return DGLGraph((u, v))

In [ ]:
G = build_graph()
print('number of nodes is :', G.number_of_nodes())
print('number of edges is :', G.number_of_edges())

number of nodes is : 3327
number of edges is : 24862


/usr/local/lib/python3.6/dist-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    lin1 = th.nn.Linear(3703, 30)
    lin2 = th.nn.Linear(30, 6)
    self.layer1 = GINConv(lin1, 'max')
    self.layer2 = GINConv(lin2, 'max')

  def forward(self, g, features):
    x = F.relu(self.layer1(g, features))
    x = self.layer2(g, x)
    return x

net = Net()
print(net)

Net(
  (layer1): GINConv(
    (apply_func): Linear(in_features=3703, out_features=30, bias=True)
  )
  (layer2): GINConv(
    (apply_func): Linear(in_features=30, out_features=6, bias=True)
  )
)


In [ ]:
def load_data():
    data = dataset
    features = th.FloatTensor(data['features'])
    labels = th.LongTensor(data['labels'])
    train_mask = th.BoolTensor(data['train_mask'])
    val_mask = th.BoolTensor(data['val_mask'])
    g = build_graph()
    return g, features, labels, train_mask, val_mask

In [ ]:
def test(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [ ]:
import time
import numpy as np
g, features, labels, train_mask, val_mask = load_data()

def train():
  optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
  dur = []
  for epoch in range(50):
      if epoch >=3:
          t0 = time.time()
      net.train()
      logits = net(g, features)
      logp = F.log_softmax(logits, 1)
      loss = F.nll_loss(logp[train_mask], labels[train_mask])

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if epoch >=3:
          dur.append(time.time() - t0)

      acc = test(net, g, features, labels, val_mask)
      print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
              epoch, loss.item(), acc, np.mean(dur)))
  save_checkpoint("best_model.pth", net)

train()


/usr/local/lib/python3.6/dist-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)
/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Loss 1.7946 | Test Acc 0.3004 | Time(s) nan
Epoch 00001 | Loss 1.7442 | Test Acc 0.4012 | Time(s) nan
Epoch 00002 | Loss 1.6718 | Test Acc 0.5123 | Time(s) nan
Epoch 00003 | Loss 1.5969 | Test Acc 0.5432 | Time(s) 0.3734
Epoch 00004 | Loss 1.5242 | Test Acc 0.5514 | Time(s) 0.3717
Epoch 00005 | Loss 1.4535 | Test Acc 0.5576 | Time(s) 0.3699
Epoch 00006 | Loss 1.3821 | Test Acc 0.5967 | Time(s) 0.3681
Epoch 00007 | Loss 1.3088 | Test Acc 0.6440 | Time(s) 0.3672
Epoch 00008 | Loss 1.2346 | Test Acc 0.6914 | Time(s) 0.3683
Epoch 00009 | Loss 1.1610 | Test Acc 0.7202 | Time(s) 0.3684
Epoch 00010 | Loss 1.0902 | Test Acc 0.7346 | Time(s) 0.3682
Epoch 00011 | Loss 1.0226 | Test Acc 0.7407 | Time(s) 0.3679
Epoch 00012 | Loss 0.9574 | Test Acc 0.7531 | Time(s) 0.3675
Epoch 00013 | Loss 0.8938 | Test Acc 0.7634 | Time(s) 0.3668
Epoch 00014 | Loss 0.8321 | Test Acc 0.7675 | Time(s) 0.3669
Epoch 00015 | Loss 0.7727 | Test Acc 0.7613 | Time(s) 0.3673
Epoch 00016 | Loss 0.7163 | Test 

In [ ]:
def save_checkpoint(checkpoint_path, model):
    # state_dict: a Python dictionary object that:
    # - for a model, maps each layer to its parameter tensor;
    state = {'state_dict': model.state_dict()}
    th.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)

In [ ]:
def load_checkpoint(checkpoint_path, model):
    state = th.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    print('model loaded from %s' % checkpoint_path)

In [ ]:
load_checkpoint("best_model.pth", net)
accuracy = test(net, g, features, labels, val_mask)
# accuracy = test(net, g, features, labels, test_mask)   # test_mask here
print("Testing Acc {:.4}".format(accuracy))

model loaded from best_model.pth
Testing Acc 0.749
